![](../img/330-banner.png)

# Lecture 12: Feature importances and model transparency

UBC 2024 Summer

Instructor: Mehrdad Oveisi

## Imports, LOs

### Imports

In [1]:
import os
import string
import sys
from collections import deque

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append(os.path.join(os.path.abspath(".."), "code"))
import seaborn as sns
from plotting_functions import *
from sklearn import datasets
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier
from utils import *

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

%matplotlib inline

ModuleNotFoundError: No module named 'seaborn'

### Learning outcomes 

From this lecture, students are expected to be able to:

- Interpret the coefficients of linear regression for ordinal, one-hot encoded categorical, and scaled numeric features. 
- Explain why interpretability is important in ML.
- Use `feature_importances_` attribute of `sklearn` models and interpret its output. 
- Apply SHAP to assess feature importances and interpret model predictions. 
- Explain force plot, summary plot, and dependence plot produced with shapely values.  

<br><br>

I'm using `seaborn` in this lecture for easy heatmap plotting, which is not in the course environment. You can install it as follows.

`conda install -n cpsc330 -c anaconda seaborn`

## Data 

In the first part of this lecture, we'll be using [Kaggle House Prices dataset](https://www.kaggle.com/c/home-data-for-ml-course/), the dataset we used in lecture 10. As usual, to run this notebook you'll need to download the data. Unzip the data into a subdirectory called `data`. For this dataset, train and test have already been separated. We'll be working with the train portion in this lecture. 

In [ ]:
df = pd.read_csv("../data/housing-kaggle/train.csv")
train_df, test_df = train_test_split(df, test_size=0.10, random_state=123)
train_df.head()

- The prediction task is predicting `SalePrice` given features related to properties.  
- Note that the target is numeric, not categorical.

In [ ]:
train_df.shape

### Let's separate `X` and `y`

In [ ]:
X_train = train_df.drop(columns=["SalePrice"])
y_train = train_df["SalePrice"]

X_test = test_df.drop(columns=["SalePrice"])
y_test = test_df["SalePrice"]

### Let's identify feature types

In [ ]:
drop_features = ["Id"]
numeric_features = [
    "BedroomAbvGr",
    "KitchenAbvGr",
    "LotFrontage",
    "LotArea",
    "OverallQual",
    "OverallCond",
    "YearBuilt",
    "YearRemodAdd",
    "MasVnrArea",
    "BsmtFinSF1",
    "BsmtFinSF2",
    "BsmtUnfSF",
    "TotalBsmtSF",
    "1stFlrSF",
    "2ndFlrSF",
    "LowQualFinSF",
    "GrLivArea",
    "BsmtFullBath",
    "BsmtHalfBath",
    "FullBath",
    "HalfBath",
    "TotRmsAbvGrd",
    "Fireplaces",
    "GarageYrBlt",
    "GarageCars",
    "GarageArea",
    "WoodDeckSF",
    "OpenPorchSF",
    "EnclosedPorch",
    "3SsnPorch",
    "ScreenPorch",
    "PoolArea",
    "MiscVal",
    "YrSold",
]

In [ ]:
ordinal_features_reg = [
    "ExterQual",
    "ExterCond",
    "BsmtQual",
    "BsmtCond",
    "HeatingQC",
    "KitchenQual",
    "FireplaceQu",
    "GarageQual",
    "GarageCond",
    "PoolQC",
]
ordering = [
    "Po",
    "Fa",
    "TA",
    "Gd",
    "Ex",
]  # if N/A it will just impute something, per below
ordering_ordinal_reg = [ordering] * len(ordinal_features_reg)
ordering_ordinal_reg

In [ ]:
ordinal_features_oth = [
    "BsmtExposure",
    "BsmtFinType1",
    "BsmtFinType2",
    "Functional",
    "Fence",
]
ordering_ordinal_oth = [
    ["NA", "No", "Mn", "Av", "Gd"],
    ["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    ["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    ["Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"],
    ["NA", "MnWw", "GdWo", "MnPrv", "GdPrv"],
]

In [ ]:
categorical_features = list(
    set(X_train.columns)
    - set(numeric_features)
    - set(ordinal_features_reg)
    - set(ordinal_features_oth)
    - set(drop_features)
)
categorical_features

In [ ]:
from sklearn.compose import ColumnTransformer, make_column_transformer

numeric_transformer = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())
ordinal_transformer_reg = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder(categories=ordering_ordinal_reg),
)

ordinal_transformer_oth = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder(categories=ordering_ordinal_oth),
)

categorical_transformer = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(handle_unknown="ignore", sparse_output=False),
)

preprocessor = make_column_transformer(
    ("drop", drop_features),
    (numeric_transformer, numeric_features),
    (ordinal_transformer_reg, ordinal_features_reg),
    (ordinal_transformer_oth, ordinal_features_oth),
    (categorical_transformer, categorical_features),
)

In [ ]:
preprocessor.fit(X_train)
preprocessor.named_transformers_

In [ ]:
ohe_columns = list(
    preprocessor.named_transformers_["pipeline-4"]
    .named_steps["onehotencoder"]
    .get_feature_names_out(categorical_features)
)
new_columns = (
    numeric_features + ordinal_features_reg + ordinal_features_oth + ohe_columns
)

In [ ]:
X_train_enc = pd.DataFrame(
    preprocessor.transform(X_train), index=X_train.index, columns=new_columns
)
X_train_enc

In [ ]:
X_train_enc.shape

In [ ]:
lr_pipe = make_pipeline(preprocessor, Ridge())
scores = cross_validate(lr_pipe, X_train, y_train, return_train_score=True)
pd.DataFrame(scores)

<br><br><br><br>

## Feature importances

- How does the output depend upon the input? 
- How do the predictions change as a function of a particular feature?
- If the model is bad, interpretability does not make sense. 

### SimpleFeature correlations

- Let's look at the correlations between various features with other features and the target in our encoded data (first row/column). 
- In simple terms here is how you can interpret correlations between two variables $X$ and $Y$:
  - If $Y$ goes up when $X$ goes up, we say $X$ and $Y$ are **positively correlated**.
  - If $Y$ goes down when $X$ goes up, we say $X$ and $Y$ are **negatively correlated**.
  - If $Y$ is unchanged when $X$ changes, we say $X$ and $Y$ are **uncorrelated**.

Let's examine the correlations among different columns, including the target column. 

In [ ]:
cor = pd.concat((y_train, X_train_enc), axis=1).iloc[:, :10].corr()
plt.figure(figsize=(8, 6))
sns.set(font_scale=0.8)
sns.heatmap(cor, annot=True, cmap=plt.cm.Blues);

- We can immediately see that `SalePrice` is highly correlated with `OverallQual`.
- This is an early hint that `OverallQual` is a useful feature in predicting `SalePrice`.
- However, this approach is **extremely simplistic**.
  - It only looks at **each feature in isolation**.
  - It only looks at **linear associations**:
    - What if `SalePrice` is high when `BsmtFullBath` is 2 or 3, but low when it's 0, 1, or 4? They might seem uncorrelated.

In [ ]:
cor = pd.concat((y_train, X_train_enc), axis=1).iloc[:, 10:15].corr()
plt.figure(figsize=(4, 4))
sns.set(font_scale=0.8)
sns.heatmap(cor, annot=True, cmap=plt.cm.Blues);

- Looking at this diagram also tells us the relationship between features. 
  - For example, `1stFlrSF` and `TotalBsmtSF` are highly correlated. 
  - Do we need both of them?
  - If our model says `1stFlrSF` is very important and `TotalBsmtSF` is very unimportant, do we trust those values?
  - Maybe `TotalBsmtSF` only "becomes important" if `1stFlrSF` is removed.
  - Sometimes the opposite happens: a feature only becomes important if another feature is _added_.

<br><br>

### Feature importances in linear models 

- With linear regression we can look at the _coefficients_ for each feature.
- Overall idea: predicted price = intercept + $\sum_i$ coefficient i $\times$ feature i.

In [ ]:
lr = make_pipeline(preprocessor, Ridge())
lr.fit(X_train, y_train);

Let's look at the coefficients. 

In [ ]:
lr_coefs = pd.DataFrame(
    data=lr.named_steps["ridge"].coef_, index=new_columns, columns=["Coefficient"]
)
lr_coefs.head(20)

Let's try to interpret coefficients for different types of features. 

### Interpreting coefficients of different types of features. 

### Ordinal features

- The ordinal features are easiest to interpret. 

In [ ]:
print(ordinal_features_reg)

In [ ]:
lr_coefs.loc[["ExterQual"]]

- Increasing by one category of exterior quality (e.g. good -> excellent) increases the predicted price by $\sim\$4237$.
  - Wow, that's a lot! 
  - Remember this is just what the model has learned. It doesn't tell us how the world works. 

In [ ]:
one_example = X_test[:1]

In [ ]:
one_example

In [ ]:
one_example[["ExterQual"]]

Let's perturb the example and change `ExterQual` to `Ex`. 

In [ ]:
one_example_perturbed = one_example.copy()
one_example_perturbed["ExterQual"] = "Ex"  # Change Gd to Ex

In [ ]:
one_example_perturbed[["ExterQual"]]

How does the prediction change after changing `ExterQual` from `Gd` to `Ex`? 

In [ ]:
print("Prediction on the original example: ", lr.predict(one_example))
print("Prediction on the perturbed example: ", lr.predict(one_example_perturbed))
print(
    "After changing ExterQual from Gd to Ex increased the prediction by: ",
    lr.predict(one_example_perturbed) - lr.predict(one_example),
)

That's exactly the learned coefficient for `ExterQual`! 

In [ ]:
lr_coefs.loc["ExterQual"]

So our interpretation is correct! 
- Increasing by one category of exterior quality (e.g. good -> excellent) increases the predicted price by $\sim\$4237$.

<br><br>

### Categorical features

- What about the categorical features?
- We have created a number of columns for each category with OHE and each category gets it's own coefficient. 

In [ ]:
print(categorical_features)

In [ ]:
lr_coefs_landslope = lr_coefs[lr_coefs.index.str.startswith("LandSlope")]
lr_coefs_landslope

- We can talk about switching from one of these categories to another by **picking a "reference" category**:

In [ ]:
lr_coefs_landslope.loc[["LandSlope_Gtl"]]

In [ ]:
lr_coefs_landslope - lr_coefs_landslope.loc["LandSlope_Gtl"]

- If you change the category from `LandSlope_Gtl` to `LandSlope_Mod` the prediction price goes up by $\sim\$6950$
- If you change the category from `LandSlope_Gtl` to `LandSlope_Sev` the prediction price goes down by $\sim\$8356$


Note that this might not make sense in the real world but this is what our model decided to learn given this small amount of data. 

In [ ]:
one_example = X_test[:1]
one_example[['LandSlope']]

Let's perturb the example and change `LandSlope` to `Mod`. 

In [ ]:
one_example_perturbed = one_example.copy()
one_example_perturbed["LandSlope"] = "Mod"  # Change Gtl to Mod

In [ ]:
one_example_perturbed[["LandSlope"]]

How does the prediction change after changing `LandSlope` from `Gtl` to `Mod`? 

In [ ]:
print("Prediction on the original example: ", lr.predict(one_example))
print("Prediction on the perturbed example: ", lr.predict(one_example_perturbed))
print(
    "After changing LandSlope from Gtl to Mod increased the prediction by: ",
    lr.predict(one_example_perturbed) - lr.predict(one_example),
)

Our interpretation above is correct! 

In [ ]:
lr_coefs.sort_values(by="Coefficient")

In [ ]:
# For comparison, we can see all RoofMatl one hot columns:
lr_coefs[lr_coefs.index.str.startswith("RoofMatl")].sort_values(by="Coefficient")

- For example, the above coefficient says that "If the roof is made of clay or tile, the predicted price is \\$191K less"?
- Do we believe these interpretations??
  - Do we believe this is how the predictions are being computed? Yes.
  - Do we believe that this is how the world works? No. 

___
If you did `drop='first'` in one hot encoding (we didn't) then you already have a reference class, and all the values are with respect to that one. The interpretation depends on whether we did `drop='first'`, hence the hassle.
___

<br><br>

### Interpreting coefficients of **numeric features**

Let's look at coefficients of `PoolArea`, `LotFrontage`, `LotArea`. 

In [ ]:
lr_coefs.loc[["PoolArea", "LotFrontage", "LotArea"]]

Intuition: 

- **Tricky** because numeric features are **scaled**!
- **Increasing** `PoolArea` by ***1 scaled unit*** **increases** the predicted price by $\sim\$2817$.
- **Increasing** `LotArea` by ***1 scaled unit*** **increases** the predicted price by $\sim\$5118$.
- **Increasing** `LotFrontage` by ***1 scaled unit*** **decreases** the predicted price by $\sim\$1583$.

Does that sound reasonable?

- For `PoolArea` and `LotArea`, yes. 
- For `LotFrontage`, that's surprising. Something positive would have made more sense?

It might be the case that `LotFrontage` is correlated with some other variable, which might have a larger positive coefficient (`LotArea`?). 

BTW, let's make sure the predictions behave as expected:

### Example showing how to interpret coefficients of scaled features

- What's **one scaled unit** for `LotArea`? 
- The scaler **subtracted** the mean and **divided** by the standard deviation.
- The **division** actually **changed the scale**! 
- For the **unit conversion**, we don't care about the subtraction, but only the scaling (by division).

In [ ]:
scaler = preprocessor.named_transformers_["pipeline-1"]["standardscaler"]

In [ ]:
scaler.var_

In [ ]:
lr_scales = pd.DataFrame(
    data=np.sqrt(scaler.var_), index=numeric_features, columns=["Scale"]
)
lr_scales.head()

- It seems like `LotArea` was divided by 8994.471032 sqft. 

In [ ]:
lr_coefs.loc[["LotArea"]]

- The coefficient tells us that if we increase the **scaled** `LotArea` by one scaled unit the price would go up by $\approx\$5118$. 
- **One scaled unit** represents $\sim 8994$ sqft in the original scale. 

Let's examine whether this behaves as expected. 

In [ ]:
X_test_enc = pd.DataFrame(
    preprocessor.transform(X_test), index=X_test.index, columns=new_columns
)

In [ ]:
one_ex_preprocessed = X_test_enc[:1]
one_ex_preprocessed

In [ ]:
orig_pred = lr.named_steps["ridge"].predict(one_ex_preprocessed.values)
orig_pred

In [ ]:
one_ex_preprocessed_perturbed = one_ex_preprocessed.copy()
one_ex_preprocessed_perturbed["LotArea"] += 1  # we are adding one to the scaled LotArea
one_ex_preprocessed_perturbed

We are expecting an increase of $5118.03516073 in the prediction compared to the original value of `LotArea`. 

In [ ]:
perturbed_pred = lr.named_steps["ridge"].predict(one_ex_preprocessed_perturbed.values)

In [ ]:
perturbed_pred - orig_pred

Our interpretation is correct! 

- Humans find it easier to think about features in their original scale.  
- How can we interpret this coefficient in the original scale? 
- If I increase original `LotArea` by one square foot then the predicted price would go up by this amount: 

In [ ]:
5118.035161 / 8994.471032 # Coefficient learned on the scaled features / the scaling factor for this feature

In [ ]:
lr_coefs.loc[["LotArea"]]

In [ ]:
lr_scales.loc[["LotArea"]]

That is, if we increase original `LotArea` by one square foot then the predicted price would go up by $0.57

Let's examine whether this works as expected. In other words 

In [ ]:
one_example = X_test[:1]

In [ ]:
one_example

Let's **perturb** the example and add 1 to the `LotArea`. 

In [ ]:
one_example_perturbed = one_example.copy()
one_example_perturbed["LotArea"] += 1

if we add 8994.471032 to the original LotArea, the housing price prediction should go up by the coefficient 5109.35671794. 

In [ ]:
one_example_perturbed

Prediction on the original example. 

In [ ]:
lr.predict(one_example)

Prediction on the perturbed example. 

In [ ]:
lr.predict(one_example_perturbed)

- What's the difference between predictions? 
- Does the difference make sense given the coefficient of the feature? 

In [ ]:
lr.predict(one_example_perturbed) - lr.predict(one_example)

Yes! Our interpretation is correct. 

- That said don't read too much into these coefficients without statistical training. 

### Interim summary

- **Correlation** among features might make coefficients completely **uninterpretable**. 
- Fairly **straightforward** to interpret coefficients of **ordinal** features. 
- In **categorical** features, it's often helpful to consider **one category as a reference** point and think about relative importance. 
- For **numeric** features, relative importance is meaningful **after scaling**.
  - You have to be careful about the scale of the feature when interpreting the coefficients. 
- Remember that explaining the model $\neq$ explaining the data or explaining how the world works.
  - The **coefficients** tell us only about the **model** and they might **not** accurately reflect the **data**. 

<br><br><br><br>

## Transparency and explainability of ML models: Motivation 

### Why model transparency/interpretability? 

- Ability to interpret ML models is crucial in many applications such as banking, healthcare, and criminal justice.
- It can be leveraged by domain experts to diagnose systematic errors and underlying biases of complex ML systems. 

![](../img/shap_example.png)

<!-- <img src="img/shap_example.png" width="600" height="600"> -->
    
[Source](https://github.com/slundberg/shap/blob/master/docs/presentations/February%202018%20Talk.pptx)

### What is model interpretability? 

- In this lecture, our definition of model interpretability will be looking at **feature importances**, i.e., exploring features which are important to the model. 
- There is more to interpretability than feature importances, but it's a good start!
- Resources: 
    - [Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/interpretability-importance.html)
    - [Yann LeCun, Kilian Weinberger, Patrice Simard, and Rich Caruana: Panel debate on interpretability](https://vimeo.com/252187813)

<br><br>

### Data

- Let's work with [the adult census data set](https://www.kaggle.com/uciml/adult-census-income) from the last lecture. 

In [ ]:
adult_df_large = pd.read_csv("../data/adult.csv")
train_df, test_df = train_test_split(adult_df_large, test_size=0.2, random_state=42)
train_df_nan = train_df.replace("?", np.NaN)
test_df_nan = test_df.replace("?", np.NaN)
train_df_nan.head()

In [ ]:
numeric_features = ["age", "capital.gain", "capital.loss", "hours.per.week"]
categorical_features = [
    "workclass",
    "marital.status",
    "occupation",
    "relationship",
    "native.country",
]
ordinal_features = ["education"]
binary_features = ["sex"]
drop_features = ["race", "education.num", "fnlwgt"]
target_column = "income"

In [ ]:
education_levels = [
    "Preschool",
    "1st-4th",
    "5th-6th",
    "7th-8th",
    "9th",
    "10th",
    "11th",
    "12th",
    "HS-grad",
    "Prof-school",
    "Assoc-voc",
    "Assoc-acdm",
    "Some-college",
    "Bachelors",
    "Masters",
    "Doctorate",
]

In [ ]:
assert set(education_levels) == set(train_df["education"].unique())

In [ ]:
numeric_transformer = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())
tree_numeric_transformer = make_pipeline(SimpleImputer(strategy="median"))

categorical_transformer = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(handle_unknown="ignore"),
)

ordinal_transformer = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OrdinalEncoder(categories=[education_levels], dtype=int),
)

binary_transformer = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(drop="if_binary", dtype=int),
)

preprocessor = make_column_transformer(
    ("drop", drop_features),
    (numeric_transformer, numeric_features),
    (ordinal_transformer, ordinal_features),
    (binary_transformer, binary_features),
    (categorical_transformer, categorical_features),
)

In [ ]:
X_train = train_df_nan.drop(columns=[target_column])
y_train = train_df_nan[target_column]

X_test = test_df_nan.drop(columns=[target_column])
y_test = test_df_nan[target_column]

In [ ]:
# encode categorical class values as integers for XGBoost
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train_num = label_encoder.fit_transform(y_train)
y_test_num = label_encoder.transform(y_test)

### Do we have class imbalance? 

- There is class imbalance. But without any context, **both classes seem equally important**. 
- Let's use accuracy as our metric. 

In [ ]:
train_df_nan["income"].value_counts(normalize=True)

In [ ]:
scoring_metric = "accuracy"

Let's store all the results in a dictionary called `results`. 

In [ ]:
results = {}

We are going to use models outside sklearn. Some of them cannot handle categorical target values. So we'll convert them to integers using `LabelEncoder`. 

In [ ]:
y_train_num

### Baseline

In [ ]:
dummy = DummyClassifier()
results["Dummy"] = mean_std_cross_val_scores(
    dummy, X_train, y_train_num, return_train_score=True, scoring=scoring_metric
)

### Different models

In [ ]:
from lightgbm.sklearn import LGBMClassifier
from xgboost import XGBClassifier

pipe_lr = make_pipeline(
    preprocessor, LogisticRegression(max_iter=2000, random_state=123)
)
pipe_rf = make_pipeline(preprocessor, RandomForestClassifier(random_state=123))
pipe_xgb = make_pipeline(
    preprocessor, XGBClassifier(random_state=123, eval_metric="logloss", verbosity=0)
)
pipe_lgbm = make_pipeline(preprocessor, LGBMClassifier(random_state=123, verbose=-1))
classifiers = {
    "logistic regression": pipe_lr,
    "random forest": pipe_rf,
    "XGBoost": pipe_xgb,
    "LightGBM": pipe_lgbm,
}

In [ ]:
for (name, model) in classifiers.items():
    results[name] = mean_std_cross_val_scores(
        model, X_train, y_train_num, return_train_score=True, scoring=scoring_metric
    )

In [ ]:
pd.DataFrame(results).T

- Logistic regression is giving reasonable scores but not the best ones. 
- XGBoost and LightGBM are giving us the best CV scores. 

- Often simple models (e.g., linear models) are interpretable but not very accurate.
- Complex models (e.g., LightGBM) are more accurate but less interpretable. 

![](../img/shap_motivation.png)

<!-- <img src="img/shap_motivation.png" width="500" height="500"> -->

[Source](https://github.com/slundberg/shap/blob/master/docs/presentations/February%202018%20Talk.pptx)

### Feature importances in linear models

Let's create and fit a pipeline with preprocessor and logistic regression. 

In [ ]:
pipe_lr = make_pipeline(preprocessor, LogisticRegression(max_iter=2000, random_state=2))
pipe_lr.fit(X_train, y_train_num);

In [ ]:
ohe_feature_names = (
    pipe_rf.named_steps["columntransformer"]
    .named_transformers_["pipeline-4"]
    .named_steps["onehotencoder"]
    .get_feature_names_out(categorical_features)
    .tolist()
)
feature_names = (
    numeric_features + ordinal_features + binary_features + ohe_feature_names
)
# feature_names[:15]

In [ ]:
pd.DataFrame(columns=feature_names)

In [ ]:
data = {
    "coefficient": pipe_lr.named_steps["logisticregression"].coef_.flatten().tolist(),
    "magnitude": np.absolute(
        pipe_lr.named_steps["logisticregression"].coef_.flatten().tolist()
    ),
}
coef_df = pd.DataFrame(data, index=feature_names).sort_values(
    "magnitude", ascending=False
)

In [ ]:
coef_df[:10]

- Increasing `capital.gain` is likely to push the prediction towards ">50k" income class 
- Whereas occupation of private house service is likely to push the prediction towards "<=50K" income. 

Can we get feature importances for non-linear models? 

<br><br>

## Model interpretability beyond linear models

- We will be looking at interpretability in terms of feature importances. 
- Note that there is no absolute or perfect way to get feature importances. But it's useful to get some idea on feature importances. So we just try our best. 

We will be looking at two ways to get feature importances. 

- `sklearn`'s `feature_importances_` and `permutation_importance`
- [SHAP](https://github.com/slundberg/shap)    

### `sklearn`'s `feature_importances_` and `permutation_importance`

**Feature importance** or **variable importance** is a score associated with a feature which tells us how "important" the feature is to the model.

### Activity (~5 mins)

Linear models learn a coefficient associated with each feature which tells us the importance of the feature to the model. 
- What might be some reasonable ways to calculate feature importances of the following models? 
    - Decision trees    
    - Linear SVMs
    - KNNs, RBF SVMs
- Suppose you have correlated features in your dataset. Do you need to be careful about this when you examine feature importances? 

### Do we have correlated features? 

In [ ]:
X_train_enc = preprocessor.fit_transform(X_train).todense()
corr_df = pd.DataFrame(X_train_enc, columns=feature_names).corr().abs()

In [ ]:
corr_df[corr_df == 1] = 0 # Set the diagonal to 0. 

- Let's look at columns where any correlation number is > 0.80. 
- 0.80 is an arbitrary choice

In [ ]:
high_corr = [column for column in corr_df.columns if any(corr_df[column] > 0.80)]
print(high_corr)

Seems like there are some columns which are highly correlated.  

In [ ]:
corr_df['occupation_missing']['workclass_missing']

In [ ]:
corr_df['marital.status_Married-civ-spouse']['relationship_Husband']

- When we look at the feature importances, we should be mindful of these correlated features. 
- Remember the limitations of looking at simple linear correlations. 
- You should probably investigate multi-colinearity with more sophisticated approaches.   

### `sklearn`'s `feature_importances_` attribute vs `permutation_importance`

- Feature importances can be 
    - algorithm dependent, i.e., calculated based on the information given by the model algorithm (e.g., gini importance) 
    - model agnostic (e.g., by measuring increase in prediction error after permuting feature values). 
    
- Different measures give insight into different aspects of your data and model. 

> [Here](https://scikit-learn.org/stable/modules/permutation_importance.html#relation-to-impurity-based-importance-in-trees) you will find some drawbacks of using `feature_importances_` attribute  in the context of tree-based models.  

#### Decision tree feature importances 

In [ ]:
pipe_dt = make_pipeline(preprocessor, DecisionTreeClassifier(max_depth=3))
pipe_dt.fit(X_train, y_train_num);

In [ ]:
data = {
    "Importance": pipe_dt.named_steps["decisiontreeclassifier"].feature_importances_,
}
pd.DataFrame(data=data, index=feature_names,).sort_values(
    by="Importance", ascending=False
)[:10]

In [ ]:
custom_plot_tree(pipe_dt.named_steps["decisiontreeclassifier"], feature_names = feature_names, fontsize=10)

Let's explore permutation importance. 
- For each feature this method evaluates the impact of permuting feature values 

In [ ]:
from sklearn.inspection import permutation_importance
def get_permutation_importance(model):
    X_train_perm = X_train.drop(columns=["race", "education.num", "fnlwgt"])
    result = permutation_importance(model, X_train_perm, y_train_num, n_repeats=10, random_state=123)
    perm_sorted_idx = result.importances_mean.argsort()
    plt.boxplot(
        result.importances[perm_sorted_idx].T,
        vert=False,
        labels=X_train_perm.columns[perm_sorted_idx],
    )
    plt.xlabel('Permutation feature importance')
    plt.show()

In [ ]:
get_permutation_importance(pipe_dt)

Decision tree is primarily making all decisions based on three features: marital.status, education, and capital.gain. 

Let's create and fit a pipeline with preprocessor and random forest. 

#### Random forest feature importances

In [ ]:
pipe_rf = make_pipeline(preprocessor, RandomForestClassifier(random_state=2))
pipe_rf.fit(X_train, y_train_num);

Which features are driving the predictions the most? 

In [ ]:
data = {
    "Importance": pipe_rf.named_steps["randomforestclassifier"].feature_importances_,
}
rf_imp_df = pd.DataFrame(
    data=data,
    index=feature_names,
).sort_values(by="Importance", ascending=False)

In [ ]:
rf_imp_df[:8]

In [ ]:
np.sum(pipe_rf.named_steps["randomforestclassifier"].feature_importances_)

In [ ]:
get_permutation_importance(pipe_rf)

Random forest is using more features in the model compared to decision trees. 

### Key point 

- Unlike the linear model coefficients, `feature_importances_` **do not have a sign**!
  - They tell us about **importance**, but ***not*** an "**up** or **down**".
  - Indeed, increasing a feature may cause the prediction to first go up, and then go down.
  - This cannot happen in linear models, because they are linear. 

### How can we get feature importances for non `sklearn` models? 

- One way to do it is by using a tool called [`eli5`](https://eli5.readthedocs.io/en/latest/overview.html).
  - Unfortunately, this is **not compatible** with the latest version of sklearn, which we are using. 

`conda install -n cpsc330 -c conda-forge eli5`

<br>

- Another popular way is using `SHAP`. You can install it using the following in the course conda environment. 

`conda install -n cpsc330 -c conda-forge shap`

<br><br><br><br>

## SHAP  (SHapley Additive exPlanations) introduction

### Explaining a prediction 

![](../img/shap_example.png)
<!-- <img src="img/shap_example.png" width="600" height="600"> -->

[Source](https://github.com/slundberg/shap/blob/master/docs/presentations/February%202018%20Talk.pptx)

### SHAP  (SHapley Additive exPlanations)
- Based on the idea of shapely values. A shapely value is created for each example and each feature. 
- Can explain the prediction of an example by computing the contribution of each feature to the prediction. 
- Great visualizations 
- Support for different kinds of models; fast variants for tree-based models
- Original paper: [Lundberg and Lee, 2017](https://arxiv.org/pdf/1705.07874.pdf)

#### Our focus
- How to use it on our dataset?
- How to generate and interpret plots created by SHAP? 
- We are not going to discuss how SHAP works. 

![](../img/shap_explanation2.png)
<!-- <img src="img/shap_explanation2.png" width="1000" height="1000"> -->

[Source](https://github.com/slundberg/shap/blob/master/docs/presentations/February%202018%20Talk.pptx)


- Start at a base rate (e.g., how often people get their loans rejected).
- Add one feature at a time and see how it impacts the decision. 

### SHAP on LGBM model 

- Let's try it out on our best performing LightGBM model. 
- You should have `shap` in the course conda environment

Let's create train and test dataframes with our transformed features. 

In [ ]:
X_train_enc = pd.DataFrame(
    data=preprocessor.transform(X_train).toarray(),
    columns=feature_names,
    index=X_train.index,
)
X_train_enc.head()

In [ ]:
X_test_enc = pd.DataFrame(
    data=preprocessor.transform(X_test).toarray(),
    columns=feature_names,
    index=X_test.index,
)
X_test_enc.shape

Let's get SHAP values for train and test data. 

In [ ]:
import shap

# Create a shap explainer object 
pipe_lgbm.named_steps["lgbmclassifier"].fit(X_train_enc, y_train)
lgbm_explainer = shap.TreeExplainer(pipe_lgbm.named_steps["lgbmclassifier"])
train_lgbm_shap_values = lgbm_explainer.shap_values(X_train_enc)

In [ ]:
train_lgbm_shap_values

- For each example, each feature, and each class we have a SHAP value.
- SHAP values tell us how to fairly distribute the prediction among features. 
- For classification it's a bit confusing. It gives SHAP matrix for all classes.
- Let's stick to shap values for class 1, i.e., income > 50K. 

In [ ]:
train_lgbm_shap_values[1].shape

In [ ]:
test_lgbm_shap_values = lgbm_explainer.shap_values(X_test_enc)
test_lgbm_shap_values[1].shape

- You can think of this as **global feature importance**
- But we'll see next that **it gives you much more**

<br><br>

## SHAP plots

In [ ]:
# load JS visualization code to notebook
shap.initjs()

### Force plots

- Most useful! 
- Let's try to explain predictions on a couple of examples from the test data. 
- I'm sampling some examples where target is <=50K and some examples where target is >50K. 

In [ ]:
y_test_reset = y_test.reset_index(drop=True)
y_test_reset

In [ ]:
l50k_ind = y_test_reset[y_test_reset == "<=50K"].index.tolist()
g50k_ind = y_test_reset[y_test_reset == ">50K"].index.tolist()

ex_l50k_index = l50k_ind[10]
ex_g50k_index = g50k_ind[10]

### Explaining a prediction

Imagine that you are given the following test example. 

In [ ]:
X_test_enc.iloc[ex_l50k_index]

You get the following hard prediction, which you are interested in explaining.  

In [ ]:
pipe_lgbm.named_steps["lgbmclassifier"].predict(X_test_enc)[ex_l50k_index]

You can first look at `predict_proba` output to get a better understanding of model confidence. 

In [ ]:
pipe_lgbm.named_steps["lgbmclassifier"].predict_proba(X_test_enc)[ex_l50k_index]

- The model seems quite confident. But if we want to know more, for example, which feature values are playing a role in this specific prediction, we can use SHAP force plots. 
- Remember that we have SHAP values per feature per example. We'll use these values to create SHAP force plot.  

In [ ]:
pd.DataFrame(
    test_lgbm_shap_values[1][ex_l50k_index, :],
    index=feature_names,
    columns=["SHAP values"],
)

SHAP will produce the following type of plots.  

In [ ]:
shap.force_plot(
    lgbm_explainer.expected_value[1], # expected value for class 1. 
    test_lgbm_shap_values[1][ex_l50k_index, :], # SHAP values associated with the example we want to explain
    X_test_enc.iloc[ex_l50k_index, :], # Feature vector of the example 
    matplotlib=True,
)

- The raw model score is much smaller than the base value, which is reflected in the prediction of <= 50k class. 
- sex = 1.0, scaled age = 0.48 are pushing the prediction towards higher score. 
- education = 8.0, occupation_Other-service = 1.0 and marital.status_Married-civ-spouse = 0.0 are pushing
the prediction towards lower score. 

In [ ]:
pipe_lgbm.named_steps["lgbmclassifier"].classes_

We can get the raw model output by passing `raw_score=True` in `predict`. 

In [ ]:
pipe_lgbm.named_steps["lgbmclassifier"].predict(X_test_enc, raw_score=True)

What's the raw score of the example above we are trying to explain?  

In [ ]:
pipe_lgbm.named_steps["lgbmclassifier"].predict(X_test_enc, raw_score=True)[ex_l50k_index]

- The score matches with what we see in the force plot. 
- The base score above is the mean raw score. Our example has a lower raw score compared to the average raw score and the force plot tries to explain which feature values are bringing this score to a lower value. 

In [ ]:
pipe_lgbm.named_steps["lgbmclassifier"].predict(X_train_enc, raw_score=True).mean()

In [ ]:
lgbm_explainer.expected_value[1]  # on average this is the raw score for class 1

Note: a nice thing about SHAP values is that the feature importances sum to the prediction:

In [ ]:
test_lgbm_shap_values[1][ex_l50k_index, :].sum() + lgbm_explainer.expected_value[1]

<br><br>

Now let's try to explain another prediction. 
- The hard prediction here is 1. 
- From the `predict_proba` output it seems like the model is not too confident about the prediction. 

In [ ]:
pipe_lgbm.named_steps["lgbmclassifier"].predict(X_test_enc)[ex_g50k_index]

In [ ]:
# X_test_enc.iloc[ex_g50k_index]

In [ ]:
pipe_lgbm.named_steps["lgbmclassifier"].predict_proba(X_test_enc)[ex_g50k_index]

What's the raw score for this example?

In [ ]:
pipe_lgbm.named_steps["lgbmclassifier"].predict(X_test_enc, raw_score=True)[
    ex_g50k_index
]  # raw model score

In [ ]:
# pd.DataFrame(
#     test_lgbm_shap_values[1][ex_g50k_index, :],
#     index=feature_names,
#     columns=["SHAP values"],
# )

In [ ]:
shap.force_plot(
    lgbm_explainer.expected_value[1],
    test_lgbm_shap_values[1][ex_g50k_index, :],
    X_test_enc.iloc[ex_g50k_index, :],
    matplotlib=True,
)

Observations: 

- Everything is with respect to class 1 here. 
- The base value, i.e., the average raw score for class 1 is -2.336. 
- We see the forces that drive the prediction. 
- That is, we can see the main factors pushing it from the base value (average over the dataset) to this particular prediction. 
- Features that push the prediction to a higher value are shown in red. 
- Features that push the prediction to a lower value are shown in blue.

<br><br>

### Global feature importance using SHAP

Let's look at the average SHAP values associated with each feature. 

In [ ]:
values = np.abs(train_lgbm_shap_values[1]).mean(
    0
)  # mean of shapely values in each column
pd.DataFrame(data=values, index=feature_names, columns=["SHAP"]).sort_values(
    by="SHAP", ascending=False
)[:10]

### Dependence plot

In [ ]:
import numpy
numpy.version.version

In [ ]:
shap.dependence_plot("age", train_lgbm_shap_values[1], X_train_enc)

The plot above shows effect of `age` feature on the prediction. 

- Each dot is a single prediction for examples above.
- The x-axis represents values of the feature age (scaled).
- The y-axis is the SHAP value for that feature, which represents how much knowing that feature's value changes the output of the model for that example's prediction. 
- Lower values of age have smaller SHAP values for class ">50K".
- Similarly, higher values of age also have a bit smaller SHAP values for class ">50K", which makes sense.  
- There is some optimal value of age between scaled age of 1 which gives highest SHAP values for for class ">50K". 
- Ignore the colour for now. The color corresponds to a second feature (education feature in this case) that may have an interaction effect with the feature we are plotting. 

### Summary plot 

In [ ]:
shap.summary_plot(train_lgbm_shap_values[1], X_train_enc)

The plot shows the most important features for predicting the class. It also shows the direction of how it's going to drive the prediction.  

- Presence of the marital status of Married-civ-spouse seems to have bigger SHAP values for class 1 and absence seems to have smaller SHAP values for class 1. 
- Higher levels of education seem to have bigger SHAP values for class 1 whereas smaller levels of education have smaller SHAP values. 

In [ ]:
shap.summary_plot(train_lgbm_shap_values[1], X_train_enc, plot_type="bar")

You can think of this as global feature importances.

<br><br>

Here, we explore SHAP's TreeExplainer. It also provides explainer for different kinds of models. 

- [TreeExplainer](https://shap.readthedocs.io/en/latest/) (supports XGBoost, CatBoost, LightGBM) 
- [DeepExplainer](https://shap.readthedocs.io/en/latest/index.html#shap.DeepExplainer) (supports deep-learning models)
- [KernelExplainer](https://shap.readthedocs.io/en/latest/index.html#shap.KernelExplainer) (supports kernel-based models)
- [GradientExplainer](https://shap.readthedocs.io/en/latest/index.html#shap.GradientExplainer) (supports Keras and Tensorflow models)

- Can also be used to explain text classification and image classification 
- Example: In the picture below, red pixels represent positive SHAP values that increase the probability of the class, while blue pixels represent negative SHAP values the reduce the probability of the class. 

![](../img/shap_image_explainer.png)
<!-- <img src="img/shap_image_explainer.png" width="600" height="600"> -->
    
[Source](https://github.com/slundberg/shap)

### Other tools

- [lime](https://github.com/marcotcr/lime) is another package.

If you're not already impressed, keep in mind:

- So far we've only used sklearn models.
- Most sklearn models have some built-in measure of feature importances.
- On many tasks we need to move beyond sklearn, e.g. LightGBM, deep learning.
- These tools work on other models as well, which makes them extremely useful.

#### Why do we want this information?

Possible reasons:

- Identify features that are not useful and maybe remove them.
- Get guidance on what new data to collect.
  - New features related to useful features -> better results.
  - Don't bother collecting useless features -> save resources.
- Help explain why the model is making certain predictions. 
  - Debugging, if the model is behaving strangely.
  - Regulatory requirements.
  - Fairness / bias. See [this](https://shap.readthedocs.io/en/latest/example_notebooks/overviews/Explaining%20quantitative%20measures%20of%20fairness.html). 
  - Keep in mind this can be used on **deployment** predictions!

Here are some guidelines and important points to remember when you work on a prediction problem where you also want to understand which features are influencing the predictions.  

- Examine multicoliniarity in your dataset using methods such as VIF. 
- If you observe high correlations in your dataset, either get rid of redundant features or be mindful of these correlations during interpretation. 
- Be mindful that feature relevance is not clearly defined. Adding/removing features can change feature importance/unimportance. Also, feature importances do not give us causal relationships. See [this optional section](https://pages.github.ubc.ca/mds-2022-23/DSCI_573_feat-model-select_students/lectures/04_feat-importances-selection.html#optional-problems-with-feature-selection) from Lecture 4. 
- Most of the models we use in ML are regularized models. With L2 regularization, the feature importances are distributed evenly among correlated features. With L1 regularization, one of the correlated features gets a high importance and the other gets a lower importance.
- Don't be overconfident. Always take feature importance values with a grain of salt.  